In [1]:
import pandas as pd
from pandas.tseries.offsets import MonthEnd
import numpy as np
import pmdarima as pm
from pmdarima.model_selection import train_test_split
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA
from multiprocessing import cpu_count
from datetime import datetime, timedelta

c:\Users\andre\AppData\Local\Programs\Python\Python310\lib\site-packages\statsforecast\core.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [5]:
sales = pd.read_csv('../data/sales_products_ts.csv')
products_info = pd.read_csv("../data/products_statistics.csv")


In [84]:
def split(df):
  weeks_train = round((max(df['ds']) - min(df['ds'])).days / 7 * 0.8)
  split_date = min(df['ds']) + timedelta(weeks=weeks_train)
  train = df[df['ds']< split_date] 
  test = df[df['ds'] >= split_date]
  return split_date, train, test

In [6]:
def preprocessing(df):
    sorted = df.groupby('id').sum('quantity').sort_values('quantity', ascending=False)
    id_list = sorted.index.unique()
    df = df.rename(columns={'date':'ds', 'quantity':'y'})
    df = df.set_index('id')
    df = df.rename_axis('unique_id')
    df.ds = pd.to_datetime(df.ds)
    df = df.sort_values(by=['unique_id', 'ds'])
    df = df[['ds','y']]
    df = df.drop_duplicates()
    return df, id_list


In [7]:
#sales = sales[sales['quantity']!=0]
sales, id_list = preprocessing(sales)

In [ ]:
train_complete = []
for item in id_list:
  one_product = sales[sales.index==item]
  split_date, train, test = split(one_product)
  print(train)
  #train_complete = pd.concat([train_complete, train])


In [ ]:
fcst = StatsForecast(
    sales,
    models = [AutoARIMA(season_length = 4)],
    freq = 'W',
    n_jobs = min(len(sales.index.unique()),cpu_count())
)

#%time arima_fc = fcst.forecast(12)

In [27]:
m = sales.loc[['17410306', '20410301']]

Arima: weekly, seasonality mensual

In [28]:
sf = StatsForecast(
    models = [AutoARIMA(season_length = 4)],
    freq = 'W',
    n_jobs = min(len(sales.index.unique()),cpu_count())
)
sf.fit(m)
forecast_df = sf.predict(h=12, level=[90]) 

In [25]:
m["ds"]=pd.to_datetime(m["ds"])
sf.plot(m, forecast_df, level=[90])

In [ ]:
round(mape(test.quantity,test.qtyMean),2)

In [ ]:
df = pd.read_csv('https://datasets-nixtla.s3.amazonaws.com/air-passengers.csv')
sf = StatsForecast(
    models = [AutoARIMA(season_length = 12)],
    freq = 'M'
)

sf.fit(df)
forecast_df = sf.predict(h=12, level=[90]) 
df["ds"]=pd.to_datetime(df["ds"])
sf.plot(df, forecast_df, level=[90])